In [ ]:
%pip install yfinance pandas numpy matplotlib pyarrow

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import datetime
from hull_moving_average_concavity import calculate_hma_signals
from smoothed_heikin_ashi import calculate_smoothed_heikin_ashi
from apply_labels import apply_labels

In [ ]:
data = yf.download('TQQQ', start='2010-01-01', end='2024-01-01')
vti_data = yf.download('VTI', start='2010-01-01', end='2024-01-01')

vti_data['15d_SMA'] = vti_data['Close'].rolling(window=15).mean()
vti_data['190d_SMA'] = vti_data['Close'].rolling(window=190).mean()

# Calculate the MACD and Signal Line
# MACD Line: 12-day EMA - 26-day EMA
# Signal Line: 9-day EMA of the MACD Line
data['12d_EMA'] = data['Close'].ewm(span=12, adjust=False).mean()
data['26d_EMA'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['12d_EMA'] - data['26d_EMA']
data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Initialize columns for Buy/Sell signals and Trailing Stop Loss
data['Buy_Signal'] = False
data['Sell_Signal'] = False
data['Trailing_Max'] = np.nan
data['In_Position'] = False

# Define the trading strategy
trailing_stop_loss = 0.4  # 15% trailing stop loss

""" for i in range(1, len(data)):
    # Update Trailing Max if in position
    if data['In_Position'][i-1]:
        data['Trailing_Max'][i] = max(data['Trailing_Max'][i-1], data['High'][i])
    
    # Check for Buy Signal
    if data['MACD'][i] > data['Signal_Line'][i] and data['MACD'][i] > 0 and not data['In_Position'][i-1]:
        data['Buy_Signal'][i] = True
        data['In_Position'][i] = True
        data['Trailing_Max'][i] = data['High'][i]
 """
for i in range(1, len(data)):
    if data['In_Position'][i-1]:
        data['Trailing_Max'][i] = max(data['Trailing_Max'][i-1], data['High'][i])

    # Check for Buy Signal based on VTI SMA crossover
    if vti_data['15d_SMA'][i] > vti_data['190d_SMA'][i] and vti_data['15d_SMA'][i-1] <= vti_data['190d_SMA'][i-1]:
        data['Buy_Signal'][i] = True
        data['In_Position'][i] = True
        data['Trailing_Max'][i] = data['High'][i]
        
    # Check for Sell Signal
    if data['In_Position'][i-1]:
        if data['Low'][i] < data['Trailing_Max'][i] * (1 - trailing_stop_loss):
            data['Sell_Signal'][i] = True
            data['In_Position'][i] = False
        else:
            data['In_Position'][i] = data['In_Position'][i-1]

# Calculate the returns
data['Returns'] = data['Close'].pct_change()
data['Strategy_Returns'] = 0
data.loc[data['Buy_Signal'], 'Strategy_Returns'] = data['Returns'][data['Buy_Signal']]
data.loc[data['Sell_Signal'], 'Strategy_Returns'] = -data['Returns'][data['Sell_Signal']]
cumulative_returns = (1 + data['Strategy_Returns']).cumprod() - 1

In [ ]:
# Plotting the strategy performance using Plotly
fig = go.Figure()

# Add TQQQ price trace
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='TQQQ Price'))

# Add Buy signals
buy_signals = data[data['Buy_Signal']]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', 
                         marker=dict(color='green', size=10, symbol='triangle-up')))

# Add Sell signals
sell_signals = data[data['Sell_Signal']]
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', 
                         marker=dict(color='red', size=10, symbol='triangle-down')))

fig.update_layout(title='TQQQ Trading Strategy Performance', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [2]:
data = yf.download('QQQ', start='2020-01-01').reset_index() 


c:\Users\Robin\Desktop\test\.venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
data = calculate_smoothed_heikin_ashi(data)
data = calculate_hma_signals(data)
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()

In [10]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['haOpen'], 
                high=data['haHigh'],
                low=data['haLow'], 
                close=data['haClose'])])
fig.update_layout(title='Heikin-Ashi Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [8]:
fig = go.Figure()

color_map = {
    'dark_orange': '#FF8C00',  # or any CSS color of your choice
    'red': 'red',
    'dark_green': 'green',  # Adjust as needed to match CSS color names or use hex codes
    'green': '#00FF00',
    'none': 'none'  # Handle 'none' as a special case if needed
}

# Update the loop to use the color_map for converting color names
for i in range(len(data) - 1):
    color = color_map[data['color'].iloc[i]] if data['color'].iloc[i] in color_map else 'none'
    fig.add_trace(go.Scatter(x=[data['Date'].iloc[i], data['Date'].iloc[i + 1]],
                             y=[data['HMA'].iloc[i], data['HMA'].iloc[i + 1]],
                             mode='lines',
                             line=dict(color=color, width=2),
                             showlegend=False))
# Additional options to customize the plot
fig.update_layout(title='HMA Concavity',
                  xaxis_title='Date',
                  yaxis_title='Price')  # Using dark theme for better visibility of colors

# Show the plot
fig.show()

In [9]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['haOpen'], 
                                     high=data['haHigh'],
                                     low=data['haLow'], 
                                     close=data['haClose'],
                                     name='Heikin-Ashi')])

fig.add_trace(go.Scatter(x=data['Date'],
                        mode='lines',
                        line=dict(width=2),
                        y=data['Close'],
                        name='QQQ Raw'))

# Define color map for HMA line
color_map = {
    'dark_orange': '#FF8C00',  
    'red': 'red',
    'dark_green': 'green',  
    'green': '#00FF00',
    'none': 'none' 
}

# Overlay the HMA line with color coding for concavity
for i in range(len(data) - 1):
    color = color_map[data['color'].iloc[i]] if data['color'].iloc[i] in color_map else 'none'
    fig.add_trace(go.Scatter(x=[data['Date'].iloc[i], data['Date'].iloc[i + 1]],
                             y=[data['HMA'].iloc[i], data['HMA'].iloc[i + 1]],
                             mode='lines',
                             line=dict(color=color, width=2),
                             showlegend=False,
                             name='HMA'))

# Customize the layout
fig.update_layout(title='Stock Price with Heikin-Ashi and HMA Concavity',
                  xaxis_title='Date',
                  yaxis_title='Price')
fig.show()

In [25]:
def apply_strategy(df):
    """
    Apply the trading strategy to the DataFrame and add an 'Alert' column with the strategy alerts.
    """
    # Determine market condition
    bear_market = (df['Close'] < df['EMA_50']) & (df['Close'] < df['EMA_200'])
    bull_market = (df['Close'] > df['EMA_50']) & (df['Close'] > df['EMA_200'])
    sideways_market = (df['Close'] < df['EMA_50']) & (df['Close'] > df['EMA_200'])
    
    df.loc[bear_market, 'Market_Condition'] = 'Bear'
    df.loc[bull_market, 'Market_Condition'] = 'Bull'
    df.loc[sideways_market, 'Market_Condition'] = 'Sideways'

    # Bear market conditions
    long_alert_bear = bear_market & (df['HA_Color'] == 'green') & (df['HMA_color'].isin(['green', 'dark_green']))
    sell_alert_bear = bear_market & ((df['HMA_color'].isin(['dark_orange', 'red'])) | (df['HA_Color'] == 'red'))

    long_alert_sideways = sideways_market & (df['HA_Color'] == 'green') & (df['HMA_color'].isin(['green', 'dark_green']))
    sell_alert_sideways = sideways_market & ((df['HMA_color'].isin(['dark_orange', 'red'])) | (df['HA_Color'] == 'red'))
    
    # Bull market conditions
    long_alert_bull = bull_market & (df['HA_Color'] == 'green') & (df['HMA_color'].isin(['green', 'dark_green']))
    sell_alert_bull = bull_market & ((df['HMA_color'] == 'red') | (df['HA_Color'] == 'red'))
    orange_hma_alert_bull = bull_market & (df['HMA_color'] == 'dark_orange')

    # Apply alerts
    df.loc[long_alert_bear, 'Alert'] = 'Long Alert (Bear)'
    df.loc[sell_alert_bear, 'Alert'] = 'Sell Alert (Bear)'
    df.loc[long_alert_sideways, 'Alert'] = 'Long Alert (Sideways)'
    df.loc[sell_alert_sideways, 'Alert'] = 'Sell Alert (Sideways)'
    df.loc[long_alert_bull, 'Alert'] = 'Long Alert (Bull)'
    df.loc[sell_alert_bull, 'Alert'] = 'Sell Alert (Bull)'
    df.loc[orange_hma_alert_bull, 'Alert'] = 'Orange HMA Alert (Bull)'

    return df

# Apply the refactored strategy function to the data
data_with_alerts = apply_strategy(data)

In [7]:

data_with_alerts = apply_labels(data)

In [26]:
data_with_alerts.to_csv("test2.csv")

In [ ]:
# implement alerts for strategy:
# use latest day data
# when below 50 EMA and 200 EMA (bear)
# long alert at first green Heikin-ashi candle/dark green or green HMA, sell alert at first orange HMA or first red HA candle
# when above 50 EMA and 200 EMA (bull)
# long at first green Heikin-ashi candle/dark green or green HMA, sell at red HMA or first red HA candle but mark and send alert when green HMA becomes orange
# above 50 EMA and below 200 EMA is sideways market
# apply bear market rules but also note sideways